In [202]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.utils import class_weight
import multiprocessing as mp
import calendar
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import ast

In [203]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        

In [204]:
def contains_european(x):
    y = x.lower()
    flag = 'e.u' in y or 'eu' in y or 'european' in y or 'european union' in y or 'europeanunion' in y
    return flag

In [217]:
year = 2015
bbc_id = 54
exclude = 'no_Others'

In [218]:
df = pd.read_csv('./data/partition_predictions/topics_pred_on_bert_partitioned_bbc_{}_{}_with_news_classifier_{}.csv'.format(bbc_id, year, exclude))
df = df.drop(['Unnamed: 0'], axis=1)
df['topic'] = df['topic'].apply(lambda x: ast.literal_eval(x))
df['european_union'] = df['transcript'].apply(lambda x: contains_european(x))

In [219]:
df['topic'] = df['topic'].apply(lambda x: x[0][0])
df

,partition_id,date,transcript,topic,european_union
0,0,5-jul-2015,# You’re stayin’ alive Stayin’ alive # Feel th...,"Parliament, government and politics",False
1,0,5-jul-2015,The polls have closed in Greece’s crucial refe...,"Parliament, government and politics",True
2,0,5-jul-2015,"Also on the programme, as he prepares for the ...","Parliament, government and politics",False
3,0,5-jul-2015,Princess Charlotte has been christened in a pr...,"Culture, media and sport",False
4,0,5-jul-2015,"And the reigning world champion, after a bad s...","Culture, media and sport",False
...,...,...,...,...,...
55268,1360,25-jan-2015,"Football now, and Match of the Day - with Spor...","Culture, media and sport",False
55269,1360,25-jan-2015,"Back to our main story, the Greek collections ...","Parliament, government and politics",True
55270,1360,25-jan-2015,I need to know how this economy will turn arou...,"Parliament, government and politics",False
55271,1360,25-jan-2015,Syriza up has promised much and after years of...,"Parliament, government and politics",True


In [220]:
df['partition_id'].value_counts().mean()

40.61204996326231

In [221]:
n_bbc = df['partition_id'].max()
n_bbc

1360

In [222]:
topics = list(topics_name_to_index_map.keys())
header = ['day', 'month']
header += topics
rows = []

In [223]:
for month in range(1, 13):
    for day in range(1, calendar.monthrange(year, month)[1]): # calendar.monthrange(year, month)[1]
        date_string = '{}-{}-{}'.format(day, calendar.month_abbr[month].lower(), year)
        print(date_string)
        df_day = df.loc[df.date == date_string]
#         df_day = df_day.loc[df.partition_id == 1259]
        partition_id = None
        pos = 0
        topic_ranks_for_day = {topic:[] for topic in topics}
        last_topic = None
#         display(df_day)
        for index, row in df_day.iterrows():
            if partition_id is None:
                partition_id = row['partition_id']
                pos = 1
                last_topic = row['topic']
            elif partition_id == row['partition_id']:
                if last_topic == row['topic']:
                    continue
                last_topic = row['topic']
                pos += 1
            else:
                pos = 1
                partition_id = row['partition_id']
                last_topic = row['topic']
            topic_ranks_for_day[row['topic']].append(pos)
#         print(topic_ranks_for_day)
        row = [day, month]
        row += [np.mean(topic_ranks_for_day[topic]) for topic in topics]
        rows.append(row)

1-jan-2015
2-jan-2015
3-jan-2015
4-jan-2015
5-jan-2015
6-jan-2015
7-jan-2015
8-jan-2015
9-jan-2015
10-jan-2015
11-jan-2015
12-jan-2015
13-jan-2015
14-jan-2015
15-jan-2015
16-jan-2015


/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


17-jan-2015
18-jan-2015
19-jan-2015
20-jan-2015
21-jan-2015
22-jan-2015
23-jan-2015
24-jan-2015
25-jan-2015
26-jan-2015
27-jan-2015
28-jan-2015
29-jan-2015
30-jan-2015
1-feb-2015
2-feb-2015
3-feb-2015
4-feb-2015
5-feb-2015
6-feb-2015
7-feb-2015
8-feb-2015
9-feb-2015
10-feb-2015
11-feb-2015
12-feb-2015
13-feb-2015
14-feb-2015
15-feb-2015
16-feb-2015
17-feb-2015
18-feb-2015
19-feb-2015
20-feb-2015
21-feb-2015
22-feb-2015
23-feb-2015
24-feb-2015
25-feb-2015
26-feb-2015
27-feb-2015
1-mar-2015
2-mar-2015
3-mar-2015
4-mar-2015
5-mar-2015
6-mar-2015
7-mar-2015
8-mar-2015
9-mar-2015
10-mar-2015
11-mar-2015
12-mar-2015
13-mar-2015
14-mar-2015
15-mar-2015
16-mar-2015
17-mar-2015
18-mar-2015
19-mar-2015
20-mar-2015
21-mar-2015
22-mar-2015
23-mar-2015
24-mar-2015
25-mar-2015
26-mar-2015
27-mar-2015
28-mar-2015
29-mar-2015
30-mar-2015
1-apr-2015
2-apr-2015
3-apr-2015
4-apr-2015
5-apr-2015
6-apr-2015
7-apr-2015
8-apr-2015
9-apr-2015
10-apr-2015
11-apr-2015
12-apr-2015
13-apr-2015
14-apr-2015
15-apr-

In [224]:
res = pd.DataFrame(rows, columns=header)

In [225]:
res

,day,month,"Agriculture, animals, food and rural affairs","Asylum, immigration and nationality","Business, industry and consumers",Communities and families,"Crime, civil law, justice and rights","Culture, media and sport",Defence,Economy and finance,...,European Union,Health services and medicine,Housing and planning,International affairs,"Parliament, government and politics",Science and technology,Social security and pensions,Social services,Transport,Others
0,1,1,NaN,NaN,NaN,NaN,4.000000,7.888889,NaN,NaN,...,NaN,9.000000,NaN,6.416667,8.066667,NaN,NaN,NaN,7.000000,NaN
1,2,1,16.0,NaN,25.000000,NaN,3.000000,13.727273,NaN,27.0,...,NaN,11.666667,NaN,14.000000,13.181818,NaN,NaN,NaN,14.937500,NaN
2,3,1,NaN,NaN,NaN,NaN,8.666667,8.333333,8.0,NaN,...,NaN,4.333333,NaN,7.500000,7.888889,NaN,NaN,NaN,7.750000,NaN
3,4,1,NaN,NaN,11.333333,NaN,10.000000,8.333333,NaN,3.0,...,NaN,7.500000,NaN,14.000000,8.562500,NaN,NaN,NaN,7.625000,NaN
4,5,1,NaN,NaN,12.666667,NaN,10.000000,12.611111,NaN,5.0,...,NaN,7.166667,NaN,10.400000,10.714286,NaN,NaN,NaN,12.571429,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,26,12,37.0,NaN,9.800000,13.0,NaN,53.523810,NaN,NaN,...,NaN,50.600000,NaN,56.466667,67.877551,NaN,NaN,NaN,54.366667,NaN
349,27,12,NaN,NaN,NaN,NaN,11.000000,9.750000,NaN,NaN,...,NaN,5.000000,NaN,8.000000,9.041667,NaN,NaN,NaN,10.000000,NaN
350,28,12,NaN,NaN,NaN,NaN,9.666667,6.909091,17.0,NaN,...,NaN,NaN,NaN,13.000000,7.500000,NaN,NaN,NaN,8.000000,NaN
351,29,12,NaN,NaN,NaN,NaN,NaN,8.000000,8.5,NaN,...,NaN,NaN,NaN,8.000000,6.937500,NaN,NaN,NaN,6.800000,NaN


In [226]:
res.to_csv('partition_topic_ranks_by_day_month_{}_{}.csv'.format(bbc_id, year))

In [227]:
res2 = res.groupby(['month']).agg('mean').drop(['day'], axis=1)
res2

,"Agriculture, animals, food and rural affairs","Asylum, immigration and nationality","Business, industry and consumers",Communities and families,"Crime, civil law, justice and rights","Culture, media and sport",Defence,Economy and finance,Education,Employment and training,...,European Union,Health services and medicine,Housing and planning,International affairs,"Parliament, government and politics",Science and technology,Social security and pensions,Social services,Transport,Others
month,,,,,,,,,,,,,,,,,,,,,
1,15.283333,NaN,14.964448,3.000000,10.708648,13.272429,12.333333,12.411765,12.333333,31.000000,...,2.000000,11.374339,NaN,11.381188,11.571224,NaN,NaN,NaN,13.606936,NaN
2,43.121795,NaN,27.381401,NaN,19.021875,23.473971,21.164966,28.789352,12.353125,9.583333,...,84.500000,23.836343,NaN,21.051403,22.090826,NaN,NaN,NaN,25.526027,NaN
3,66.566667,NaN,41.848794,39.333333,24.330872,31.005555,39.784461,20.689014,17.412037,21.875000,...,100.200000,29.485098,NaN,33.891495,31.148380,NaN,NaN,NaN,35.907252,NaN
4,40.206111,NaN,23.933473,6.000000,20.140368,23.139227,24.822222,13.668821,17.220000,NaN,...,41.272727,24.513872,NaN,23.108851,21.837138,NaN,NaN,NaN,25.178136,NaN
5,29.187778,NaN,23.509900,16.500000,12.709708,15.713046,22.500000,32.355000,23.692308,11.500000,...,27.333333,15.974901,NaN,15.354949,15.220217,NaN,NaN,NaN,16.032113,NaN
6,39.970833,NaN,22.116239,NaN,14.705553,19.298056,29.319444,8.035000,14.837662,15.166667,...,19.673611,21.129464,NaN,16.564999,17.603849,NaN,NaN,NaN,20.722571,NaN
7,33.319444,NaN,20.728788,12.000000,18.400554,17.842433,20.124359,22.370370,16.159091,NaN,...,25.777778,18.714787,NaN,15.914617,15.985860,NaN,NaN,NaN,16.687618,NaN
8,17.428571,NaN,16.794372,14.916667,13.024533,15.456019,12.630000,12.142857,18.348485,20.500000,...,12.500000,13.919444,NaN,12.909093,13.961516,NaN,NaN,NaN,15.653169,NaN
9,33.060606,NaN,19.914112,NaN,17.131408,17.285018,26.031667,37.750000,17.754545,NaN,...,39.681818,27.095801,NaN,15.464920,16.093427,NaN,NaN,NaN,19.148918,NaN


In [228]:
res2.to_csv('partition_topic_ranks_by_month_{}_{}.csv'.format(bbc_id, year))